# Decorators

By the end of this session, you will know what decorators are, how to create and use them.

## So what is a decorator anyway?
By definition, a decorator takes another function and *extends* its behaviour without explicitly modifying it.

Although this may sounds confusing at first, it's really not - it will all make sense once you see some examples of how decorators work.

## Functions
**Recall**: a function returns a value based in the given parameters. 

Functions may also have side effects other than turning an input into an output. For example, the print() function returns None, while having the side effect of printing something to the console. 

In order to understand decorators, it's enough to think of functions as something that return a value.

## First Class objects
In Python, functions are first-class objects, meaning they can be passed as parameters. just like any other objects.

In [1]:
import random

def greeter(name):
    possible_greetings = ("Hello", "Yo", "Whazzaaa", "'Sup", "Servus", "Cf", "Bună ziua", "Hello darkness")
    print("{}, {}".format(random.choice(possible_greetings), name))

def say_hello_to_my_little_friend(greeter_function):
    return(greeter_function("my old friend"))

In the example above, the `greeter()` function is a regular function, that expects one argument.

The `say_hello_to_my_little_friend()` function expects a function as an argument.

Let's try passing the `greeter()` function as a parameter. What do you think will happen?

In [8]:
say_hello_to_my_little_friend(greeter)

Hello darkness, my old friend


Note that `say_hello_to_my_little_friend(greeter)` refers to two functions, but in different ways:
- `greeter` is named without parantheses, meaning that only a *reference* to the function is passed; the function is not executed
- `say_hello_to_my_little_friend()` is written with parantheses, so it will be called as usual

## Inner functions
It is possible to define functions *inside other functions*.

In [13]:
def song():
    print("Verse: If I was an astronaut, I'd be floating in mid-air")
    print("Verse: And a broken heart would just belong")

    def chorus():
        print("Chorus: I'm up in space, man")
        print("Chorus: Up in space, man")

    def bridge():
        print("Bridge: Gravity keeps pulling me down")
        print("Bridge: As long as you're on the ground, I'll stick around")

    def outro():
        print("Outro: I've searched around the universe")
        print("Outro: Been down some black holes")

    chorus()
    bridge()
    chorus()
    outro()

# what happens when you call the song() function?
# does the order in which the inner functions are defined matter?

# song()

song.chorus()

AttributeError: 'function' object has no attribute 'chorus'

The inner functions do not exist until the parent function is called; they only exist in the scope of the parent function, as local variables.

## Returning functions from functions
Python allows you to use functions as return values. 

In [22]:
def song(part):

    def chorus():
        print("Chorus: I'm up in space, man")
        print("Chorus: Up in space, man")

    def bridge():
        print("Bridge: Gravity keeps pulling me down")
        print("Bridge: As long as you're on the ground, I'll stick around")

    def outro():
        print("Outro: I've searched around the universe")
        print("Outro: Been down some black holes")

    if part == "bridge":
        return bridge
    elif part == "chorus":
        return chorus
    elif part == "outro":
        return outro
    else:
        return None

Note that the functions are returned without parantheses, meaning you are returning **a reference to the function**.

In [23]:
bridge = song("bridge")

print(type(bridge))

# as a reference to the inner function was passed, the bridge() function may be used outside of the song() function
bridge()

<class 'function'>
Bridge: Gravity keeps pulling me down
Bridge: As long as you're on the ground, I'll stick around


In [24]:
# destroy the song() function
del song

In [25]:
# does the bridge() function still work?
bridge()

Bridge: Gravity keeps pulling me down
Bridge: As long as you're on the ground, I'll stick around


## Simple decorators

You have now seen that functions are just like any other object in Python.

You are now ready to step into the magical realm of **PYTHON DECORATORS**.

Let's start with a simple example to illustrate the typical decorator behaviour.

In [26]:
def anon_greeter():
    possible_greetings = ("Hello.", "Yo.", "Whazzaaa.", "'Sup.", "Servus.", "Cf.", "Bună ziua.", "Good day.")
    print(random.choice(possible_greetings))

def greeter_decorator(func):
    def wrapper():
        print("Something happening before the greeting.")
        func()
        print("Something happening after the greeting.")
    return wrapper


decorated_greeter = greeter_decorator(anon_greeter)

You may name the inner `wrapper()` function whatever you like, it is just an ordinary inner function.

What do you think will happen whe you call the `decorated_greeter()` function?

Let's try it out!

In [31]:
decorated_greeter()

Something happening before the greeting.
Bună ziua.
Something happening after the greeting.


What happened?!

In a nutshell, **decorators wrap a function, modifying its behaviour**.

`wrapper()` is a regular function, therefore the way a decorator modifies a function can change dynamically.

Let's add a little complexity to the decorator - giving the greeter a lunch break;

In [32]:
from datetime import datetime

def greeter_decorator(func):
    def wrapper():
        print("Something happening before the greeting.")
        if datetime.now().hour == 12:
            print("on lunch break, bye")
        else:
            func()
        print("Something happening after the greeting.")
    return wrapper

decorated_greeter = greeter_decorator(anon_greeter)
decorated_greeter()

Something happening before the greeting.
on lunch break, bye
Something happening after the greeting.


## Syntactic Sugar
The way you decorated the greeter above is not the most aestetically pleasing, is it?

```
decorated_greeter = greeter_decorator(anon_greeter)
decorated_greeter()
```
Could you **be** writing "greeter" more times?! So chunky.

Luckily, there is a more aestethic way of decorating your functions: using **the @ symbol**, sometimes called the "pie" syntax.

In [33]:
# destroy the greeters
del anon_greeter
del decorated_greeter

In [34]:
# recreate the anon_greeter(), this time decorated

@greeter_decorator
def anon_greeter():
    possible_greetings = ("Hello.", "Yo.", "Whazzaaa.", "'Sup.", "Servus.", "Cf.", "Bună ziua.", "Good day.")
    print(random.choice(possible_greetings))

# this does the same thing as the previous example - the decorator is applied to the function

anon_greeter()

Something happening before the greeting.
on lunch break, bye
Something happening after the greeting.


## Decorating functions with arguments
But what if the function you want to decorate has arguments? Can you still decorate it?

Let' see.

In [35]:
@greeter_decorator
def greeter(name):
    possible_greetings = ("Hello", "Yo", "Whazzaaa", "'Sup", "Servus", "Cf", "Bună ziua", "Hello darkness")
    print("{}, {}".format(random.choice(possible_greetings), name))

In [36]:
greeter("my old friend")

TypeError: wrapper() takes 0 positional arguments but 1 was given

Oh, no, we got an error... 

`TypeError: greeter_decorator.<locals>.wrapper() takes 0 positional arguments but 1 was given`

The problem here is that the inner decorator function does not take arguments, but the `greeter()` function has one passed to it.
You can fix this by letting the `wrapper()` function accept one argument, however, that would create a problem for the previously `anon_greeter()` decorated function.
The solution is to use `*args, **kwargs` in the inner wrapper function; they will accept an arbitrary number of arguments.

**Recall** the unpacking operators * and **.

In [37]:
def greeter_decorator(func):
    def wrapper(*args, **kwargs):
        print("Something happening before the greeting.")
        if datetime.now().hour == 12:
            print("on lunch break, bye")
        else:
            func(*args, **kwargs)
        print("Something happening after the greeting.")
    return wrapper

In [38]:
@greeter_decorator
def greeter(name):
    possible_greetings = ("Hello", "Yo", "Whazzaaa", "'Sup", "Servus", "Cf", "Bună ziua", "Hello darkness")
    print("{}, {}".format(random.choice(possible_greetings), name))

In [39]:
greeter("my old friend")

Something happening before the greeting.
on lunch break, bye
Something happening after the greeting.


## Exercise

Have a look at the decorators under resources/methods and update them. Once you do that, use them in the snippet below.

In [43]:
## call the function twice
from resources.methods.decorators import do_twice

@do_twice
def repeat_after_me():
    print("I like repeating myself.")

print(repeat_after_me())

I like repeating myself.
I like repeating myself.
None


In [ ]:
## use a timer on this function
from ... import ...

def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

waste_some_time(200)

In [ ]:
## use another decorator to slow down the above function
## can you use multiple decorators on the same function?
## does the order of the decorators matter?
from ... import ....

def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

waste_some_time(200)

## Further reading
- [Python decorator wiki](https://wiki.python.org/moin/PythonDecorators)
- [Python decorator library](https://wiki.python.org/moin/PythonDecoratorLibrary)